In [ ]:
import numpy as np
import scipy.signal as signal
import scipy.fft as fft
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import csv

In [ ]:
def envelope(consData):
  #isolates daily amplitude and offset of power consumption data
  envlength = int(len(consData)/24)
  envbounds = np.zeros((2, envlength), float)
  for i in range(envlength):
    envbounds[0,i] = max(et[24*i:24*(i+1):1])
    envbounds[1,i] = min(et[24*i:24*(i+1):1])
  envamp = envbounds[0,:] - envbounds[1,:]
  envoffs = 0.5*(envbounds[0,:] + envbounds[1,:])
  return envamp, envoffs

In [121]:
with open('/content/drive/MyDrive/Colab Notebooks/ercot02_22.csv') as ercotData:
  #reads in power consumption data from csv
  #csv being used for prototyping is compilation of all ERCOT power consumption
  #data from 2002-2022
  #this csv is compiled from excel spreadsheets downloaded from their site and
  #modified slightly for ease of use
  reader = csv.reader(ercotData, quoting=csv.QUOTE_NONNUMERIC)
  data = list(reader)
  ercot_array = np.array(data)

In [122]:
#n represents integer number designating each timestamp so I don't have to deal
#with dates while prototyping
n = ercot_array[:,0]
#coast
co = ercot_array[:, 1]
#east
es = ercot_array[:, 2]
#far west
fw = ercot_array[:, 3]
#north
no = ercot_array[:, 4]
#north central
nc = ercot_array[:, 5]
#southern
so = ercot_array[:, 6]
#south central
sc = ercot_array[:, 7]
#west
we = ercot_array[:, 8]
#ercot (total combined data for all areas monitored by ERCOT)
et = ercot_array[:, 9]

In [123]:
#Isolating the amplitude and offset for each data array
coamp, cooffs = envelope(co)
esamp, esoffs = envelope(es)
fwamp, fwoffs = envelope(fw)
noamp, nooffs = envelope(no)
ncamp, ncoffs = envelope(nc)
soamp, sooffs = envelope(so)
scamp, scoffs = envelope(sc)
weamps, weoffs = envelope(we)
etamp, etoffs = envelope(et)


In [124]:
#Calculating the short-time fourier transform (STFT) of each data array.
#Window size is equal to 30 days, with 15 days worth of overlap.
#This overlap is to avoid issues with artifacting at the edges of each window.
#The amount of overlap is chosen to ensure that the signals can be reconstructed
#using the Inverse Short-Time Fourier Transform (ISTFT).
#The two print statements check whether these conditions are met.
cof, cot, coZxx = signal.stft(et, 24, nperseg=720, noverlap=360)
esf, est, esZxx = signal.stft(es, 24, nperseg=720, noverlap=360)
fwf, fwt, fwZxx = signal.stft(fw, 24, nperseg=720, noverlap=360)
nof, nort, noZxx = signal.stft(no, 24, nperseg=720, noverlap=360)
ncf, nct, ncZxx = signal.stft(nc, 24, nperseg=720, noverlap=360)
sof, sot, soZxx = signal.stft(so, 24, nperseg=720, noverlap=360)
scf, sct, scZxx = signal.stft(sc, 24, nperseg=720, noverlap=360)
wef, wet, weZxx = signal.stft(we, 24, nperseg=720, noverlap=360)
etf, ett, etZxx = signal.stft(et, 24, nperseg=720, noverlap=360)
#print(signal.check_COLA(signal.get_window("hann", 720), 720, 360))
#print(signal.check_NOLA(signal.get_window("hann", 720), 720, 360))

In [ ]:
#FOR PROTOTYPING ONLY
#This cell will be removed in the final version

#plt.pcolormesh(nct, ncf, np.abs(ncZxx), norm = "log", shading='gouraud', rasterized = True)

#yearticks = [0, 8760, 17544, 26304, 35064, 438360, 52608, 61368, 70128, 78888, 87672, 96432, 105192, 113952, 122736, 131496, 140256, 149016, 157800, 166560, 175320]
#yearlabels = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
#plt.xticks(ticks=yearticks, labels=yearlabels)
#plt.title('STFT Magnitude')

#plt.ylabel('Frequency [cycles per day]')

#plt.xlabel('Time [days]')

#plt.show